In [ ]:
import math
import numpy as np
#import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import imageio
import os
import time

In [ ]:
core = "../data/challenges/2/"
paths = []
for folder in os.listdir(core):
    if folder != 'gt' and folder != 'train':
        continue
    for subfolder in os.listdir(core + folder):
        if "." in subfolder:
            continue
        for filename in os.listdir(core + folder + '/' + subfolder):
            if not filename.endswith(".txt"):
                continue
            path = core + folder + '/' + subfolder + '/' + filename
            paths.append(path)

progress = 0
for path in paths:
    fname = path[path.rfind("/")+1:path.rfind(".")]
    progress += 1
    start_time = time.time()
    data= np.genfromtxt(path)

    axis = [np.min(data[:,2]), np.max(data[:,2]), np.min(data[:,3]), np.max(data[:,3])]
    scale = 0.7
    axis = [y-scale if y < 0 else y+scale for y in axis]

    # Regular plot of x,y positions
    plt.scatter(data[:, 2], data[:, 3])
    plt.axis(axis)
    plt.title(fname)
    #plt.title("x,y position plot")
    # plt.show()
    plt.savefig("./out/position/" + fname + ".png", bbox_inches='tight')
    plt.close()

    # Histogram of x, y positions
    # Color indicates how dense it is (in the overall dataset)
    from scipy.stats import gaussian_kde

    x = data[:, 2]
    y = data[:, 3]
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)

    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    # density 1
    plt.scatter(x, y, c=z, edgecolor='')
    plt.axis(axis)
    plt.title(fname)
#     plt.title("histogram of x,y position (color=density)")
    # plt.show()
    plt.savefig("./out/density1/" + fname + ".png", bbox_inches='tight')
    plt.close()

    # another option (density 2)
    plt.hist2d(x, y, (50, 50), cmap=plt.cm.jet)
    plt.colorbar()
    plt.title(fname)
#     plt.title("histogram of x,y position (color=density)")
    # plt.show()
    plt.savefig("./out/density2/" + fname + ".png", bbox_inches='tight')
    plt.close()

    # Pedestrian speed histogram
    # and mean speed per pedestrian
    peds = np.unique(data[:, 1])
    npeds = peds.shape[0]
    speeds = np.zeros((npeds,))

    for i in range(npeds):
        pts = data[np.where(data[:, 1] == peds[i])]
        xys = pts[:, 2:]
        vs = np.sum(np.square(xys[1:, :] - xys[:-1, :]), axis=1)
        speeds[i] = np.average(vs)

    # log
    with open("./out/notes/" + fname + ".txt", 'a') as f:
        f.write("Average speed " + str(np.average(speeds)) + "\nNumber pedestrians " + str(npeds) + "\n")

    # Mean speed per pedestrian
    plt.bar(np.arange(npeds), speeds, align='center')
#     plt.xlabel('Pedestrian')
    #plt.xticks(np.arange(npeds), peds)
    plt.title(fname)
#     plt.title("Mean speed per pedestrian")
#     plt.ylabel('Mean speed')
    # plt.show()
    plt.savefig("./out/pedestrian_speed/" + fname + ".png", bbox_inches='tight')
    plt.close()

    # Pedestrian speed histograms
    plt.hist(speeds, bins=25, edgecolor='black')
    plt.title(fname)
#     plt.title("Pedestrian speed histogram")
#     plt.xlabel('Speed')
#     plt.ylabel('# Pedestrians')
    # plt.show()
    plt.savefig("./out/overall_speed/" + fname + ".png", bbox_inches='tight')
    plt.close()

    # we can calculate for each pedestrian, speed at location
    # so make histogram where location = location, color based on that speed
    # just set z to speeds for each x,y
    # plt.scatter(x, y, c=z, edgecolor='')
    # plt.axis(axis)
    # plt.show()

    constant = -1000
    npts = data.shape[0]
    x = np.zeros((npts,))
    y = np.zeros((npts,))
    v = np.ones((npts,)) * constant

    for i in range(npts):
        x[i] = data[i, 2]
        y[i] = data[i, 3]
        if i == 0 or data[i,1] != data[i-1,1]:
            continue
        v[i] = np.square(x[i] - x[i-1]) + np.square(y[i] - y[i-1])

    inds = np.where(v != constant)
    x = x[inds]
    y = y[inds]
    v = v[inds]

    # Speed distributions in image space
    # Color indicates how fast it is
    plt.scatter(x, y, c=v, edgecolor='', cmap=plt.cm.jet, s=12, marker='s')
    plt.colorbar()
    plt.axis(axis)
    ax = plt.gca()
    ax.set_facecolor(plt.cm.jet(0.0))
    plt.title(fname)
#     plt.title("Speed dist. in image space")
    # plt.show()
    plt.savefig("./out/image_speed/" + fname + ".png", bbox_inches='tight')
    plt.close()

    # Time distributions in image space (trajectory by trajectory)
    # Color indicates frame
    # Let's create an animation/gif
    
    ns = [x for x in range(20, data.shape[0], 20)] #20 #data.shape[0] / 100

    for n in ns:
        #print n, "points"
        x = data[:n, 2]
        y = data[:n, 3]
        z = data[:n, 0]

        plt.scatter(x, y, c=z, edgecolor='', cmap=plt.cm.jet, s=12, marker='s')
        plt.colorbar()
        plt.axis(axis)
        ax = plt.gca()
        ax.set_facecolor('white')
        #plt.show()
        plt.title("Trajectories in image space over time")
        plt.savefig('./out/image_traj/' + fname + '_{}.png'.format(str(n).zfill(5)), bbox_inches='tight')
        plt.close()

    # Time distributions in image space (frame by frame)
    # Color indicates frame
    # Let's create an animation/gif

    frames = np.sort(np.unique(data[:, 0])) # sample every 5
    frames = frames[::5]
    for frame in frames:
        #break
        slices = data[np.where(data[:, 0] <= frame)]
        x = slices[:, 2]
        y = slices[:, 3]
        z = slices[:, 0]

        plt.scatter(x, y, c=z, edgecolor='', cmap=plt.cm.jet, s=12, marker='s')
        plt.colorbar()
        plt.axis(axis)
        ax = plt.gca()
        ax.set_facecolor('white')
        # plt.show()
        plt.title("Frames in image space over time")
        plt.savefig('./out/image_frame/' + fname + '_{}.png'.format(str(int(frame)).zfill(5)), bbox_inches='tight')
        plt.close()

    images = []
    for n in ns:
        filename = './out/image_traj/' + fname + '_{}.png'.format(str(n).zfill(5))
        images.append(imageio.imread(filename))
    imageio.mimsave('./out/image_traj/' + fname + '.gif', images)

    images = []
    for frame in frames:
        filename = './out/image_frame/' + fname + '_{}.png'.format(str(int(frame)).zfill(5))
        images.append(imageio.imread(filename))
    imageio.mimsave('./out/image_frame/' + fname + '.gif', images)

    for n in ns:
        filename = './out/image_traj/' + fname + '_{}.png'.format(str(n).zfill(5))
        os.remove(filename)

    for frame in frames:
        filename = './out/image_frame/' + fname + '_{}.png'.format(str(int(frame)).zfill(5))
        os.remove(filename)

    print "(" + str(progress) + "/" + str(len(paths)) + ") - ", str(int(progress*100./len(paths))) + "% -", fname, "- %s seconds" % (time.time() - start_time)